In [1]:
import math
from tqdm import tqdm
import numpy as np
import glob
import csv
import cv2
from PIL import Image

In [11]:
print("Data loading and Management")
ml_road_dir = "../ml_preds_csv/"

base_file_name = "3001120103"

# 82*8 = 656
grid = np.zeros((656, 656), dtype=np.uint8)
road_coords = dict()

level_a = {'0': (0, 0), '1': (0, 328), '2': (328, 0), '3': (328, 328)}
level_b = {'0': (0, 0), '1': (0, 164), '2': (164, 0), '3': (164, 164)}
level_c = {'0': (0, 0), '1': (0, 82), '2': (82, 0), '3': (82, 82)}
for a in tqdm(level_a):
    a_i = level_a[a][0]
    a_j = level_a[a][1]
    for b in level_b:
        b_i = a_i + level_b[b][0]
        b_j = a_j + level_b[b][1]
        for c in level_c:
            c_i = b_i + level_c[c][0]
            c_j = b_j + level_c[c][1]

            # generate files names
            file_name = base_file_name + a + b + c + ".csv"
            file_path = ml_road_dir + file_name

            with open(file_path, 'r') as csv_file:
                reader = csv.reader(csv_file)
                first_row = True
                for row in reader:
                    if first_row:
                        first_row = False
                        continue

                    d_i = int(row[0])
                    d_j = int(row[1])

                    i = c_i + int(d_i / 100)
                    j = c_j + int(d_j / 100)

                    grid[i][j] = max(np.uint8(row[2]), grid[i][j])
                    if int(row[2]) >= 75 and (i*656+j not in road_coords or road_coords[i*656+j][0] < int(row[2])):
                            road_coords[i*656+j] = (int(row[2]), float(row[3]), float(row[4]))
                            

  0%|          | 0/4 [00:00<?, ?it/s]

Data loading and Management


100%|██████████| 4/4 [08:43<00:00, 131.00s/it]


In [12]:
grid[grid < 75] = 0
img = Image.fromarray(grid)
img.show()

In [13]:
print("Image Size:", np.shape(grid))
print("total pizels: ", 656*656)
print("road pixels:", np.count_nonzero(grid))

Image Size: (656, 656)
total pizels:  430336
road pixels: 105923


In [26]:
roads = grid.copy()
roads[roads >= 75] = 1
print("Image Size:", np.shape(roads))
print("total pizels: ", 656*656)
print("road pixels:", np.count_nonzero(roads))

# visited = np.zeros(np.shape(roads))
# print("Image Size:", np.shape(visited))
# print("total pizels: ", 656*656)
# print("road pixels:", np.count_nonzero(visited))

Image Size: (656, 656)
total pizels:  430336
road pixels: 105923
Image Size: (656, 656)
total pizels:  430336
road pixels: 0


In [16]:
test_cases = []
with open("../routing_challenge_pairs.csv", 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        test_cases.append(row)
        
del test_cases[0]

In [17]:
def closest_ij(lati, lngi):
    min_dist = 999999
    i=0
    j=0
    lati = float(lati)*100000
    lngi = float(lngi)*100000
    for coord in road_coords:
        lat = float(road_coords[coord][1])*100000
        lng = float(road_coords[coord][2])*100000
        if min_dist > math.sqrt((lati - lat)**2 + (lngi - lng)**2):
            min_dist = math.sqrt((lati - lat)**2 + (lngi - lng)**2)
            i = int(coord/656)
            j = int(coord - i*656)
    return i,j

In [21]:
lat1 = test_cases[0][0]
lng1 = test_cases[0][1]
i1,j1 = closest_ij(lat1, lng1)
print("1:",(i1,j1))
print("1:", grid[i1][j1])

lat2 = test_cases[0][2]
lng2 = test_cases[0][3]
i2,j2 = closest_ij(lat2, lng2)
print("2:",(i2, j2))
print("2:", grid[i2][j2])

1: (391, 457)
1: 247
2: (562, 418)
2: 245


In [35]:
def DFS(i1, j1, i2, j2):
    visited = np.zeros(np.shape(roads))
    queue = []
    queue.append((0, i1, j1))
    visited[i1][j1] = 1
    front = 0
    kernel = [-1,0,1]
    while front != len(queue):
        (d, i, j) = queue[front]
        front += 1
        if i==i2 and j==j2:
            return d
        
        for p in kernel:
            for q in kernel:
                if 0<= i+p < 656 and 0<= j+q < 656 and roads[i+p][j+q] != 0 and visited[i+p][j+q] == 0:
                    queue.append((d+1, i+p, j+q))
                    visited[i+p][j+q] = 1
                    
    return -1

In [37]:
dist = DFS(391, 457, 562, 418)
dist*50

11200

In [38]:
(11200 - 10370)/10370

0.08003857280617165

In [41]:
expected = 0
for degree in range(0, 46, 5):
    expected += 2/math.cos(degree/180)
expected = expected / 20
expected

1.0111894476325118

In [ ]:
for case in tqdm(test_cases):
    lat1 = case[0]
    lng1 = case[1]
    i1,j1 = closest_ij(lat1, lng1)

    lat2 = case[2]
    lng2 = case[3]
    i2,j2 = closest_ij(lat2, lng2)
    
    dist = DFS(i1, j1, i2, j2) * 50
    print(dist)

  0%|          | 1/1000 [00:01<27:54,  1.68s/it]

11200


  0%|          | 2/1000 [00:02<23:59,  1.44s/it]

16100


  0%|          | 3/1000 [00:02<18:50,  1.13s/it]

...

14550


 36%|███▋      | 364/1000 [09:00<16:10,  1.53s/it]

23400


 36%|███▋      | 365/1000 [09:02<17:24,  1.64s/it]

17250
